
# Assignment 2 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 2, PROBLEM 1
Maximum Points = 8


A courier company operates a fleet of delivery trucks that make deliveries to different parts of the city. The trucks are equipped with GPS tracking devices that record the location of each truck at regular intervals. The locations are divided into three regions: downtown, the suburbs, and the countryside. The following table shows the probabilities of a truck transitioning between these regions at each time step:

| Current region | Probability of transitioning to downtown | Probability of transitioning to the suburbs | Probability of transitioning to the countryside |
|----------------|--------------------------------------------|-----------------------------------------------|------------------------------------------------|
| Downtown       | 0.3                                      | 0.4                                           | 0.3                                            |
| Suburbs        | 0.2                                      | 0.5                                           | 0.3                                            |
| Countryside    | 0.4                                      | 0.3                                           | 0.3                                            |

1. If a truck is currently in the suburbs, what is the probability that it will be in the downtown region after two time steps? [1.5p]
2. If a truck is currently in the suburbs, what is the probability that it will be in the downtown region **the first time** after two time steps? [1.5p]
3. Is this Markov chain irreducible? [1.5p]
4. What is the stationary distribution? [1.5p]
5. Advanced question: What is the expected number of steps until the first time one enters the downtown region having started in the suburbs region. Hint: to get within 1 decimal point, it is enough to compute the probabilities for hitting times below 30. [2p]



In [3]:
# Part 1

# Fill in the answer to part 1 below as a decimal number
# problem1_p1 = XXX


import numpy as np

# Transition matrix: [Downtown, Suburbs, Countryside]
P = np.array([
    [0.3, 0.4, 0.3],
    [0.2, 0.5, 0.3],
    [0.4, 0.3, 0.3]
])

# Part 1: Probability in downtown after 2 steps, starting from suburbs
# Initial state: suburbs = [0,1,0]
v0 = np.array([0, 1, 0])
v2 = v0 @ np.linalg.matrix_power(P, 2)
problem1_p1 = v2[0]  # Probability in downtown after 2 steps


In [4]:
# Part 2

# Fill in the answer to part 2 below as a decimal number
# problem1_p2 = XXX

# Part 2: Probability first time in downtown at step 2, starting from suburbs
# At step 1: not in downtown, so must be in suburbs or countryside
# At step 2: in downtown, and not in downtown at step 1
# P(suburbs->suburbs->downtown) + P(suburbs->countryside->downtown)
p_sub_sub_down = P[1,1] * P[1,0]
p_sub_coun_down = P[1,2] * P[2,0]
problem1_p2 = p_sub_sub_down + p_sub_coun_down


In [6]:
# Part 3

# Fill in the answer to part 3 below as a boolean
# problem1_irreducible = True/False

# Part 3: Is the chain irreducible?
problem1_irreducible = True  # All states communicate

In [7]:
# Part 4

# Fill in the answer to part 4 below
# the answer should be a numpy array of length 3
# make sure that the entries sums to 1!
# problem1_stationary = XXX

# Part 4: Stationary distribution
# Solve pi = pi P, sum(pi)=1
A = np.transpose(P) - np.eye(3)
A = np.vstack([A, np.ones(3)])
b = np.zeros(4)
b[3] = 1
problem1_stationary = np.linalg.lstsq(A, b, rcond=None)[0]

In [8]:
# Part 5

# Fill in the answer to part 5 below
# That is, the expected number of steps as a decimal number
# problem1_ET = XXX

# Part 5: Expected steps to downtown from suburbs (hitting time)
# Let E[i] = expected steps from state i to downtown
# E[0] = 0 (already in downtown)
# E[1] = 1 + 0.2*E[0] + 0.5*E[1] + 0.3*E[2]
# E[2] = 1 + 0.4*E[0] + 0.3*E[1] + 0.3*E[2]
# Solve linear system:
# E[0] = 0
# E[1] = 1 + 0.2*0 + 0.5*E[1] + 0.3*E[2]
# E[2] = 1 + 0.4*0 + 0.3*E[1] + 0.3*E[2]
# Rearranged:
# E[1] - 0.5*E[1] - 0.3*E[2] = 1
# E[2] - 0.3*E[1] - 0.3*E[2] = 1
# 0.5*E[1] - 0.3*E[2] = 1
# -0.3*E[1] + 0.7*E[2] = 1

A_ET = np.array([
    [0.5, -0.3],
    [-0.3, 0.7]
])
b_ET = np.array([1, 1])
E1, E2 = np.linalg.solve(A_ET, b_ET)
problem1_ET = E1  # Expected steps from suburbs to downtown

# Answers:
# problem1_p1: probability in downtown after 2 steps from suburbs
# problem1_p2: probability first time in downtown at step 2 from suburbs
# problem1_irreducible: True
# problem1_stationary: stationary distribution
# problem1_ET: expected steps from

---
## Assignment 2, PROBLEM 2
Maximum Points = 4


Use the **Multi-dimensional Constrained Optimisation** example (in `07-Optimization.ipynb`) to numerically find the MLe for the mean and variance parameter based on `normallySimulatedDataSamples`, an array obtained by a specific simulation of $30$ IID samples from the $Normal(10,2)$ random variable.

Recall that $Normal(\mu, \sigma^2)$ RV has the probability density function given by:

$$
f(x ;\mu, \sigma) = \displaystyle\frac{1}{\sigma\sqrt{2\pi}}\exp\left(\frac{-1}{2\sigma^2}(x-\mu)^2\right)
$$

The two parameters, $\mu \in \mathbb{R} := (-\infty,\infty)$ and $\sigma \in (0,\infty)$, are sometimes referred to as the location and scale parameters.

You know that the log likelihood function for $n$ IID samples from a Normal RV with parameters $\mu$ and $\sigma$ simply follows from $\sum_{i=1}^n \log(f(x_i; \mu,\sigma))$, based on the IID assumption. 

NOTE: When setting bounding boxes for $\mu$ and $\sigma$ try to start with some guesses like $[-20,20]$ and $[0.1,5.0]$ and make it larger if the solution is at the boundary. Making the left bounding-point for $\sigma$ too close to $0.0$ will cause division by zero Warnings. Other numerical instabilities can happen in such iterative numerical solutions to the MLe. You need to be patient and learn by trial-and-error. You will see the mathematical theory in more details in a future course in scientific computing/optimisation. So don't worry too much now except learning to use it for our problems.  

In [9]:
import numpy as np
from scipy import optimize
# do NOT change the next three lines
np.random.seed(123456) # set seed
# simulate 30 IID samples drawn from Normal(10,2)RV
normallySimulatedDataSamples = np.random.normal(10,2,30) 

# define the negative log likelihood function
def negLogLklOfIIDNormalSamples(parameters):
    '''return the -log(likelihood) of normallySimulatedDataSamples with mean and var parameters'''
    mu_param = parameters[0]
    sigma_param = parameters[1]
    if sigma_param <= 0:
        return np.inf  # avoid invalid sigma
    n = len(normallySimulatedDataSamples)
    x = normallySimulatedDataSamples
    # negative log likelihood for Normal(mu, sigma^2)
    neg_log_lkl = n * np.log(sigma_param * np.sqrt(2 * np.pi)) + np.sum((x - mu_param) ** 2) / (2 * sigma_param ** 2)
    return neg_log_lkl

# bounding box for (mu, sigma)
parameter_bounding_box = ((-20, 20), (0.1, 5.0))
initial_arguments = np.array([10, 2])  # initial guess: mean=10, sigma=2

result_problem2_opt = optimize.minimize(
    negLogLklOfIIDNormalSamples,
    initial_arguments,
    bounds=parameter_bounding_box,
    method='L-BFGS-B'
)

---
## Assignment 2, PROBLEM 3
Maximum Points = 4



Derive the maximum likelihood estimate for $n$ IID samples from a random variable with the following probability density function:
$$
f(x; \lambda) = \frac{1}{24} \lambda^5 x^4 \exp(-\lambda x), \qquad \text{ where, } \lambda>0, x > 0
$$

You can solve the MLe by hand (using pencil paper or using key-strokes). Present your solution as the return value of a function called `def MLeForAssignment2Problem3(x)`, where `x` is a list of $n$ input data points.

In [11]:

# do not change the name of the function, just replace XXX with the appropriate expressions for the MLe
def MLeForAssignment2Problem3(x):
    '''
    Computes the maximum likelihood estimate (MLE) for lambda
    for the given density f(x; lambda) = (1/24) * lambda^5 * x^4 * exp(-lambda * x)
    for n IID samples x.
    The MLE is: lambda_hat = 5n / sum(x)
    '''
    n = len(x)
    s = sum(x)
    lambda_hat = 5 * n / s

---
## Assignment 2, PROBLEM 4
Maximum Points = 8


## Random variable generation and transformation

The purpose of this problem is to show that you can implement your own sampler, this will be built in the following three steps:

1. [2p] Implement a Linear Congruential Generator where you tested out a good combination (a large $M$ with $a,b$ satisfying the Hull-Dobell (Thm 6.8)) of parameters. Follow the instructions in the code block.
2. [2p] Using a generator construct random numbers from the uniform $[0,1]$ distribution.
3. [4p] Using a uniform $[0,1]$ random generator, generate samples from 

$$p_0(x) = \frac{\pi}{2}|\sin(2\pi x)|, \quad x \in [0,1] \enspace .$$

Using the **Accept-Reject** sampler (**Algorithm 1** in TFDS notes) with sampling density given by the uniform $[0,1]$ distribution.

In [19]:
def problem4_LCG(size=None, seed=0):
    """
    A linear congruential generator that generates pseudo random numbers according to size.
    Uses parameters satisfying the Hull-Dobell theorem.
    """
    # Parameters: M is large, a and b satisfy Hull-Dobell
    M = 2**31 - 1  # a large prime (common choice)
    a = 16807      # Lehmer's choice, primitive root modulo M
    b = 0          # b=0 is fine for full period with this a, M

    u = seed
    out = []
    for _ in range(size):
        u = (a * u + b) % M
        out.append(u / M)
    return out

In [20]:

def problem4_uniform(generator=None, period = 1, size=None, seed=0):
    """
    Takes a generator and produces samples from the uniform [0,1] distribution according
    to size.
    
    Parameters
    -------------
    generator : a function of type generator(size,seed) and produces the same result as problem1_LCG, i.e. pseudo random numbers in the range {0,1,...,period-1}
    period : the period of the generator
    seed : the seed to be used in the generator provided
    size : an integer denoting how many samples should be produced
    
    Returns
    --------------
    out : a list of the uniform pseudo random numbers
    """
    # Use the provided generator to get raw pseudo-random numbers in {0, 1, ..., period-1}
    raw_samples = generator(size=size, seed=seed)
    # Scale to [0,1]
    out = [x / period for x in raw_samples]
    return out

In [21]:

def problem4_accept_reject(uniformGenerator=None, n_iterations=None, seed=0):
    """
    Takes a generator that produces uniform pseudo random [0,1] numbers 
    and produces samples from (pi/2)*abs(sin(x*2*pi)) using an Accept-Reject
    sampler with the uniform distribution as the proposal distribution.
    Runs n_iterations
    
    Parameters
    -------------
    generator : a function of the type generator(size,seed) that produces uniform pseudo random
    numbers from [0,1]
    seed : the seed to be used in the generator provided
    n_iterations : an integer denoting how many attempts should be made in the accept-reject sampler
    
    Returns
    --------------
    out : a list of the pseudo random numbers with the specified distribution
    """
    
    """
    Accept-Reject sampler for p0(x) = (pi/2)*|sin(2*pi*x)| on [0,1] using uniform proposal.
    """
    out = []
    # Generate 2*n_iterations uniform numbers for x and u
    uniforms = uniformGenerator(size=2 * n_iterations, seed=seed)
    # The maximum of p0(x) is pi/2 (at x=1/4, 3/4, etc.)
    c = np.pi / 2
    for i in range(n_iterations):
        x = uniforms[2 * i]
        u = uniforms[2 * i + 1]
        px = (np.pi / 2) * abs(np.sin(2 * np.pi * x))
        if u < px / c:
            out.append(x)
    return out

---
#### Local Test for Assignment 2, PROBLEM 4
Evaluate cell below to make sure your answer is valid.                             You **should not** modify anything in the cell below when evaluating it to do a local test of                             your solution.
You may need to include and evaluate code snippets from lecture notebooks in cells above to make the local test work correctly sometimes (see error messages for clues). This is meant to help you become efficient at recalling materials covered in lectures that relate to this problem. Such local tests will generally not be available in the exam.

In [22]:

# If you managed to solve all three parts you can test the following code to see if it runs
# you have to change the period to match your LCG though, this is marked as XXX.
# It is a very good idea to check these things using the histogram function in sagemath
# try with a larger number of samples, up to 10000 should run

print("LCG output: %s" % problem4_LCG(size=10, seed=1))

period = 2**31 - 1

print("Uniform sampler %s" % problem4_uniform(generator=problem4_LCG, period=period, size=10, seed=1))

uniform_sampler = lambda size, seed: problem4_uniform(generator=problem4_LCG, period=period, size=size, seed=seed)

print("Accept-Reject sampler %s" % problem4_accept_reject(uniformGenerator=uniform_sampler, n_iterations=20, seed=1))

LCG output: [7.826369259425611e-06, 0.13153778814316625, 0.7556053221950332, 0.4586501319234493, 0.5327672374121692, 0.21895918632809036, 0.04704461621448613, 0.678864716868319, 0.6792964058366122, 0.9346928959408276]
Uniform sampler [3.644437185986921e-15, 6.125205578488219e-11, 3.51856147193764e-10, 2.1357561095479172e-10, 2.4808907772426417e-10, 1.0196081662087291e-10, 2.190685655753733e-11, 3.1612101811191066e-10, 3.163220389526963e-10, 4.3525029736388376e-10]
Accept-Reject sampler [3.51856147193764e-10, 2.4808907772426417e-10, 3.163220389526963e-10, 1.785820711722791e-10, 3.869483929589921e-10, 3.1252828631073734e-10, 1.7854182558759897e-10, 1.9440705638947614e-10, 2.742636218346156e-10, 3.9402716369489213e-10, 1.937147961206251e-10, 4.2390116998625887e-10, 1.222141989443378e-10, 3.4276483809953736e-10, 2.945952900602199e-10, 4.614877449327775e-10]


In [23]:

# If however you did not manage to implement either part 1 or part 2 but still want to check part 3, you can run the code below

def testUniformGenerator(size,seed):
    import random
    random.seed(seed)
    
    return [random.uniform(0,1) for s in range(size)]

print("Accept-Reject sampler %s" % problem4_accept_reject(uniformGenerator=testUniformGenerator, n_iterations=20, seed=1))

Accept-Reject sampler [0.763774618976614, 0.651592972722763, 0.0938595867742349, 0.8357651039198697, 0.762280082457942, 0.22876222127045265, 0.9014274576114836, 0.21659939713061338, 0.23308445025757263, 0.2187810373376886, 0.28978161459048557, 0.8375779756625729, 0.6422943629324456]
